## Integrate radially first

Important: Hyperspy file needs to be calibrated and distortion corrected and beam centered

In [1]:
import warnings

from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
%matplotlib qt
import pyxem as pxm
import hyperspy.api as hs
import os, glob, tqdm
import gc
import numpy as np
from diffsims.utils.sim_utils import get_electron_wavelength

In [2]:
root = r'G:\My Drive\PhD\projects\external_measurements\ml_difsims'
folder = 'data/experimental'
file_extension = '*.hspy'

path = os.path.join(root, folder, file_extension)
paths = glob.glob(path)
paths = [p for p in paths if 'radial' not in p]
paths.sort()
paths

['G:\\My Drive\\PhD\\projects\\external_measurements\\ml_difsims\\data/experimental\\20200209_163154_centre_rebin_correct_rb_fullscan.hspy',
 'G:\\My Drive\\PhD\\projects\\external_measurements\\ml_difsims\\data/experimental\\20220126_142402_rebin_nav_2.hspy',
 'G:\\My Drive\\PhD\\projects\\external_measurements\\ml_difsims\\data/experimental\\roi_3_rebin.hspy',
 'G:\\My Drive\\PhD\\projects\\external_measurements\\ml_difsims\\data/experimental\\roi_4_rebin.hspy']

In [3]:
from pyxem.detectors import Medipix515x515Detector

# Simulation microscope values (for azimuthal integration)
beam_energy = 200.0  #keV
detector_pix_size = 55e-6  #m

for p in tqdm.tqdm(paths[1:2]):
    print(p)
    dp = hs.load(p, signal_type='electron_diffraction')

    calibration = dp.axes_manager.signal_axes[0].scale
    detector_size = dp.axes_manager.signal_axes[0].size
    radial_steps = int(np.ceil((int(detector_size / 2) - 1) / 2) * 2)

    # # Old way (pyxem way):
    #dp.set_diffraction_calibration(calibration)
    dp.metadata.Signal.ai = None
    dp.unit = "k_A^-1"
    dp.set_experimental_parameters(beam_energy=beam_energy)
    dp.set_ai(center=([detector_size / 2, detector_size / 2]))

    # New way (pyFAI)
    # detector = Medipix515x515Detector()
    # wavelength = get_electron_wavelength(beam_energy) * 1e-10
    # camera_length = detector_pix_size / (wavelength * calibration * 1e10)
    # center = ([detector_size / 2, detector_size / 2])
    # unit = "k_A^-1"
    # dp.unit = unit
    # ai = AzimuthalIntegrator(dist=camera_length, detector=detector, wavelength=wavelength)
    # ai.setFit2D(directDist=camera_length * 1000, centerX=center[1], centerY=center[0])
    # dp.metadata.set_item("Signal.ai", ai)

    radial = dp.get_azimuthal_integral1d(npt=radial_steps)
    radial2d = dp.get_azimuthal_integral2d(npt=radial_steps)
    print(radial)
    name = "{}_radial.hspy".format(os.path.basename(p).split('.')[0])
    radial.save(os.path.join(os.path.dirname(p), name), overwrite=True)
    name = "{}_radial2d.hspy".format(os.path.basename(p).split('.')[0])
    radial2d.save(os.path.join(os.path.dirname(p), name), overwrite=True)
    print(f"Finished with {name}")
    gc.collect()


  0%|          | 0/1 [00:00<?, ?it/s]

G:\My Drive\PhD\projects\external_measurements\ml_difsims\data/experimental\20220126_142402_rebin_nav_2.hspy


C:\Users\jf631\miniconda3\envs\pyxem_new\lib\site-packages\hyperspy\misc\utils.py:471: VisibleDeprecationWarning: Use of the `binned` attribute in metadata is going to be deprecated in v2.0. Set the `axis.is_binned` attribute instead. 
  warnings.warn(
C:\Users\jf631\miniconda3\envs\pyxem_new\lib\site-packages\hyperspy\io.py:560: VisibleDeprecationWarning: Loading old file version. The binned attribute has been moved from metadata.Signal to axis.is_binned. Setting this attribute for all signal axes instead.
  warnings.warn('Loading old file version. The binned attribute '


<ElectronDiffraction1D, title: , dimensions: (127, 127|160)>


ERROR:hyperspy.io_plugins._hierarchical:The writer could not write the following information in the file: ai : Detector Detector	 Spline= None	 PixelSize= 1.379e-04, 1.379e-04 m
Wavelength= 2.507934e-12m
SampleDetDist= 1.000000e+00m	PONI= 2.206982e-02, 2.206982e-02m	rot1=0.000000  rot2= 0.000000  rot3= 0.000000 rad
DirectBeamDist= 1000.000mm	Center: x=160.000, y=160.000 pix	Tilt=0.000 deg  tiltPlanRotation= 0.000 deg
Traceback (most recent call last):
  File "C:\Users\jf631\miniconda3\envs\pyxem_new\lib\site-packages\hyperspy\io_plugins\_hierarchical.py", line 774, in dict2group
    group.attrs[key] = value
  File "h5py\_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py\_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "C:\Users\jf631\miniconda3\envs\pyxem_new\lib\site-packages\h5py\_hl\attrs.py", line 103, in __setitem__
    self.create(name, data=value)
  File "C:\Users\jf631\miniconda3\envs\pyxem_new\lib\site-packages\h5py\_hl\attrs.py", line 180

Finished with 20220126_142402_rebin_nav_2_radial2d.hspy


## Crop (and rebin) to match simulated range

The simulated range has the following format. Make sure the processed exp data has the same:
- Pixel size: 147 px
- Range in q (no $2\pi/d$ but just $1/d$): (0.10777668889613681, 1.318191810345058) $\AA^{-1}$

In [4]:
# DO NOT CHANGE!!!
crop_range_q = (0.10777668889613681, 1.318191810345058) # A-1
crop_size = 147 #px

In [5]:
%matplotlib qt
import hyperspy.api as hs
import os, glob
import numpy as np
from scipy import interpolate
import warnings
import matplotlib.pyplot as plt

In [7]:
root = r'G:\My Drive\PhD\projects\external_measurements\ml_difsims'
folder = 'data/experimental'
file_extension = '*.hspy'

path = os.path.join(root, folder, file_extension)
paths = glob.glob(path)
paths = [p for p in paths if 'radial2d' in p and 'crop' not in p]
paths.sort()
paths

['G:\\My Drive\\PhD\\projects\\external_measurements\\ml_difsims\\data/experimental\\20220126_142402_rebin_nav_2_radial2d.hspy']

In [8]:
def interpolate_1d(signal_data, q_array, crop_range_q, crop_size):
    # Do interpolation
    x = q_array
    y = signal_data
    f = interpolate.interp1d(x, y, fill_value='extrapolate')

    # Generate new data
    x_new = np.linspace(crop_range_q[0], crop_range_q[1], crop_size)
    y_interpol = f(x_new)
    return y_interpol



for path in paths:
    dp = hs.load(path, signal_type='electron_diffraction')
    q_exp = dp.axes_manager.signal_axes[0].axis
    if q_exp.min() > crop_range_q[0] or q_exp.max() < crop_range_q[1]:
        warnings.warn("The range at which signal was acquired is not large enough. Extrapolation will be used using scipy.interpolate.interp1d")

    q_array = dp.axes_manager.signal_axes[0].axis
    q_new = np.linspace(crop_range_q[0], crop_range_q[1], crop_size)
    dp_crop = dp.map(interpolate_1d, q_array = q_array, crop_range_q = crop_range_q, crop_size = crop_size,
                     show_progressbar=True, parallel=True, inplace=False)

    # Correct for axes calibration
    sig_ax = dp_crop.axes_manager.signal_axes[0]
    sig_ax.offset = crop_range_q[0]
    sig_ax.scale = (crop_range_q[1] - crop_range_q[0])/ crop_size
    print(dp_crop)

    # Save files
    name = "{}_crop.hspy".format(os.path.basename(path).split('.')[0])
    dp_crop.save(os.path.join(os.path.dirname(path), name), overwrite=True)
    name = "{}_crop.npz".format(os.path.basename(path).split('.')[0])
    np.savez(os.path.join(os.path.dirname(path), 'npz_files', name), y=dp_crop.data, x=q_new)

    # Plot results
    plt.plot(q_new, dp_crop.mean().data, label=name)

plt.legend()
plt.tight_layout()

<ElectronDiffraction2D, title: , dimensions: (127, 127|147, 160)>


ValueError: x and y must have same first dimension, but have shapes (147,) and (160, 147)

In [182]:
dp_crop.mean().plot()
name

'roi_4_rebin_radial_crop.npz'

In [ ]:
dp.axes_manager, dp_crop.axes_manager

In [30]:
a = dp.data[0,0]
print(a.shape)
print(q_exp.shape)
b = np.vstack([(row.shape) for row in a])
print(b.shape)

(160, 360)
(160,)
(160, 1)


In [36]:
def interpolate_2d(signal_data, q_array, crop_range_q, crop_size):
    signal_data = signal_data.T
    print(signal_data.shape)

    y_interpol_2d = \
        np.vstack([interpolate_1d(row, q_array, crop_range_q, crop_size)
                   for row in signal_data])
    return y_interpol_2d.T


# Get the experimental q range
q_exp = dp.axes_manager.signal_axes[1].axis
print(q_exp.shape)

if q_exp.min() > crop_range_q[0] or q_exp.max() < crop_range_q[1]:
    warnings.warn(
        "The range at which signal was acquired is not large enough. Extrapolation will be used using scipy.interpolate.interp1d")

b = interpolate_2d(a, q_array=q_exp, crop_range_q=crop_range_q, crop_size=crop_size,)

(160,)
(360, 160)


C:\Users\jf631\AppData\Local\Temp\ipykernel_13868\3885628504.py:16: UserWarning: The range at which signal was acquired is not large enough. Extrapolation will be used using scipy.interpolate.interp1d
  warnings.warn(


In [37]:
plt.imshow(b)

In [39]:
plt.figure()
plt.imshow(a, vmax=0.01)

In [45]:
dp.data.max((-2, -1), keepdims=True).shape

(127, 127, 1, 1)

In [52]:
dpmax = dp.data.max((-2, -1), keepdims=True)
dpmin = dp.data.min((-2, -1), keepdims=True)
dp = (dp - dpmin) / ( dpmax - dpmin)
# Correct any nan value
nan_mask = np.isnan(dp)
dp.data[nan_mask] = 0

dp

<ElectronDiffraction2D, title: , dimensions: (127, 127|360, 160)>

In [54]:
dp.data[0,5].min()

0.0